[코드 돌릴 때 유의사항]
- 해당 코드는 어떤 과목이든 인문캠 리뷰만 조사합니다. 자연캠 과목이여도 인문캠 기준으로만 돌아갑니다.
- 코드 돌아갈 때 만약에 과목을 검색했을 때, 과목명이 나오지 않으면 바로 다음 과목으로 넘어가게끔 조치했습니다

In [1]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import requests
import pandas as pd
import time
import re
import random

In [2]:
lec_1 = pd.read_excel('교과목코드집_교양.xlsx')
lec_2 = pd.read_excel('교과목코드집_전공_대학원제거.xlsx')
merge_lec = pd.DataFrame()
merge_lec = merge_lec.append(lec_1)
merge_lec = merge_lec.append(lec_2)
merge_lec.reset_index(drop=True,inplace=True)

/var/folders/gx/q7vn3yv17rvb0vy6chbbbztw0000gn/T/ipykernel_1940/93745514.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merge_lec = merge_lec.append(lec_1)
/var/folders/gx/q7vn3yv17rvb0vy6chbbbztw0000gn/T/ipykernel_1940/93745514.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merge_lec = merge_lec.append(lec_2)


In [3]:
# merge_lec으로 얻을 수 있는 정보
lecturelist = [] # 교과목
creditlist = [] # 학점
classification_list=[] # 이론 실습 강좌유형
pnplist = [] # pass or np
gradelist = [] # 이수학년
majoryon = [] # 전공인지아닌지
majorlist = [] # 학과칼럼

# 크롤링해서 얻어야 하는 정보
proflist = [] # 교수명
reviewlist = [] # 리뷰 목록
whenlist = [] # 작성시점
scorelist = [] # 평가별 점수
votelist = [] # 공감 수

## 로그인해서 강의실 페이지 들어가기까지

In [4]:
driver = webdriver.Chrome(executable_path = 'C:/Users/chromedriver.exe')
driver.implicitly_wait(3)  # 로딩 될때까지 3초 대기
# 3초 대기
time.sleep(3)

driver.get("https://everytime.kr/lecture")
time.sleep(2)

id_bar = driver.find_element(By.CSS_SELECTOR, '#container > form > p:nth-child(1) > input')
id_bar.clear()
id_bar.send_keys('oneman98')
time.sleep(2)



pw_bar = driver.find_element(By.CSS_SELECTOR,'#container > form > p:nth-child(2) > input')
pw_bar.clear()
#비밀번호 입력해야함
pw_bar.send_keys('sinhang789@')

pw_bar.send_keys(Keys.RETURN)
time.sleep(3)

/var/folders/gx/q7vn3yv17rvb0vy6chbbbztw0000gn/T/ipykernel_1940/258489869.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = 'C:/Users/chromedriver.exe')


## 모든 과목 검색 후 강의 관련 데이터 크롤링

In [5]:
# 랜덤한 대기시간
wait_sec = (random.random()*10)%3+1

for i in range(len(merge_lec)):
    # 검색 바 찾아서 검색
    search_bar = driver.find_element(By.CSS_SELECTOR, '#container > form > input.keyword')
    search_bar.clear()
    # 과목명 입력
    search_bar.send_keys(merge_lec['교과목명'][i])
    search_bar.send_keys(Keys.RETURN)
    time.sleep(2)
    
    # 과목명
    subject = merge_lec['교과목명'][i]
    print(subject)
    credit = merge_lec['학점수'][i]
    classification = merge_lec['강좌유형'][i]
    # 패논패과목이면 o, 아니면 x로 구분
    cat = merge_lec['NP유형'][i]
    if cat=='채플' or cat=='교양NP' or cat=='사회봉사' or cat=='진로선택' or cat=='전공NP':
        pnp = "o"
    else:
        pnp = "x"
    grade = merge_lec['이수학년'][i]
    
    # 교양이라고 적혀 있으면 교양, 아니면 전공
    # ismajor : 교양인지 전공인지
    # major : 교양인지 전공인지, 단, 전공이면 어떤 과의 전공인지까지 나와있음
    if merge_lec['학과'][i] == "교양":
        ismajor = '교양'
        major = merge_lec['학과'][i]
    else:
        ismajor = "전공"
        major = merge_lec['학과'][i]
        
    # 끝까지 내리기
    a=0
    while a < 10000:
        a = a + 1
        driver.execute_script("window.scrollTo(0, 100100100100100100100100);")

    # 끝까지 올리기
    a=0
    while a < 10000:
        a = a + 1
        driver.execute_script("window.scrollTo(0, -100100100100100100100100);")
        driver.implicitly_wait(2)
        
    # 과목명 검색해서 찾기
    lectures = driver.find_elements(By.CLASS_NAME, 'lecture')
    if len(lectures)!=0:
        # 인문캠 기준으로 과목명이 존재하는지 존재하지 않는지 비교
        # 과목명이 존재하지 않는다면, 생략하고 바로 다음으로

        # 인문캠부터 수행
        for lecture in lectures:
            # 강의명, 교수명 저장
            try : 
                prof = lecture.text.split('\n')[1]
            except : 
                prof = "없음"
            # 강의 클릭하기
            lecture.click()
            # 강의평 더보기 클릭
            # 만약 강의평이 없다면, 뒤의 과정 수행은 중지하고 다음 과정으로 넘어갈 수 있게끔
            try:
                driver.find_element(By.CSS_SELECTOR, 'body > div > div > div.pane > div > section.review > div.articles > a').click()
                time.sleep(wait_sec)
            except : 
                driver.back()
                time.sleep(wait_sec+0.2)
                continue # 바로 다음 항목으로 넘어가게끔

            # 강의평 끝까지 내리기
            itemlist = driver.find_element(By.CLASS_NAME, "articles")
            b=0
            while b<10000:
                b = b + 1
                driver.execute_script("arguments[0].scrollBy(0, 100100100100100100100100)", itemlist)

            # 리뷰 담기
            reviews = driver.find_elements(By.CLASS_NAME, 'text')
            for review in reviews:
                reviewlist.append(review.text)

            # 수강학기 담기 + 교수명, 강의명 담기
            whens = driver.find_elements(By.CLASS_NAME, 'semester')

            for when in whens:
                lecturelist.append(subject) # 과목명 담기
                creditlist.append(credit) # 학점수 담기
                classification_list.append(classification) # 이론실습 강좌유형 담기
                pnplist.append(pnp) # 패논패여부 담기
                gradelist.append(grade) # 이수학년 담기
                majoryon.append(ismajor) # 전공여부
                majorlist.append(major) # 교양, 전공이면 세부과 담기

                proflist.append(prof) # 교수명 담기
                whenlist.append(when.text) # 수강학기 담기


            # 별 담기
            bs = BeautifulSoup(driver.page_source, 'html.parser')
            # # 별 개수
            bs_stars = bs.find_all('span', class_='on')
            # 1부터 한 이유는, 첫번째 즉, 0의 결과가 전체 평점이 나오기 때문
            for i in range(1, len(bs_stars)):
                score = bs_stars[i]['style'].split(': ')[1].replace(';','')
                # score에 따른 점수 할당 후 담기
                if score == '0%':
                    LecStar = 0
                    scorelist.append(LecStar)
                elif score == '20%':
                    LecStar = 1
                    scorelist.append(LecStar)
                elif score == '40%':
                    LecStar = 2
                    scorelist.append(LecStar)
                elif score == '60%':
                    LecStar = 3
                    scorelist.append(LecStar)
                elif score == '80%':
                    LecStar = 4
                    scorelist.append(LecStar)
                elif score == '100%':
                    LecStar = 5
                    scorelist.append(LecStar)

            # 추천받은 수
            votes = driver.find_elements(By.CLASS_NAME, 'info')

            for i in range(1, len(votes)):
                try:
                    vote = votes[i].text.split('\n')[1]
                    vote = int(vote)
                    votelist.append(vote)
                except:
                    vote = 0
                    votelist.append(vote)

            ## 뒤로가기 두번 누르면 원래 강의평가 페이지로 넘어가짐
            driver.back()
            time.sleep(wait_sec+0.2)
            driver.back()
            time.sleep(wait_sec+0.5)# 크롤링 감지 방지를 위해 sleep시간 다르게 주기
    else:
        time.sleep(1)
    driver.back()
    time.sleep(wait_sec+0.3)

성서와인간이해


KeyboardInterrupt: 

In [ ]:
print(len(lecturelist))
print(len(creditlist))
print(len(classification_list))
print(len(pnplist))
print(len(gradelist))
print(len(majoryon))
print(len(majorlist))
print(len(proflist))
print(len(reviewlist))
print(len(whenlist))
print(len(scorelist))
print(len(votelist))

In [ ]:
df = pd.DataFrame(zip(lecturelist, creditlist, classification_list, pnplist, gradelist, majoryon, majorlist, proflist, reviewlist, whenlist, scorelist, votelist))

df.columns = ['강의명','학점','강좌유형','패논패여부','이수학년','전공여부','학과','교수명','리뷰','작성시점','평가별점수','공감수']

In [ ]:
df.to_csv('인문캠전과목리뷰.csv', index=False, encoding='utf-8-sig')